# Lab 1: **Kernels** and **features**

Advanced Topics in Machine Learning -- Spring 2023, UniTS

<a target="_blank" href="https://colab.research.google.com/github/ganselmif/adv-ml-units/blob/main/notebooks/AdvML_UniTS_2023_Lab_01_Intro_to_Kernels.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

### Empirical verification of the *Kernel* $\leftrightarrow$ *feature expansion* equivalence

Recall the definition of *kernel*:
> Let $\mathcal{X}$ be a non-empty set. A function $k: \mathcal{X} \times \mathcal{X} \rightarrow \mathbb{R}$ is called a *kernel* if there exists a real-Hilbert space $\mathcal{H}$ and a map $\phi: \mathcal{X} \rightarrow \mathcal{H}$ such that $\forall x, x^\prime \in \mathcal{X}$, $k(x, x^\prime) := \langle \phi(x) , \phi(x^\prime) \rangle_{\mathcal{H}}$.

To motivate the usefulness of kernelized ML methods, we can show that -- for $x\in\mathbb{R}^{d \in \mathbb{N}}$ -- the computation of $k(x, x^\prime)$ in kernel form is equivalent to the explicit scalar product $\langle \varphi(x) , \varphi(x^\prime) \rangle = \varphi(x)^{T} \varphi(x^\prime)$ of some corresponding expanded feature maps $\varphi: {R}^{d} \rightarrow \mathbb{R}^{d^\prime}$ with generally $d^\prime \gg d$ (or even *infinite-dimensional* $\varphi$s), though significantly simpler and more efficient to compute.

In the lab that follows, verify such equivalence for simple kernels: the non-uniform *quadratic* (in $\mathbb{R}^{d}$) and the *Gaussian* (in $\mathbb{R}$).

For each kernel:

1. Implement a function that computes the kernel between two arrays of coordinates;
2. Derive the explicit feature map $\varphi(x)$ corresponding to that kernel;
3. Implement a function that computes such feature map for a given array of coordinates;
4. Verify that the kernel computed by (1) and the scalar product of its arguments through (3) are indeed equivalent.

**Hint**: in case of need, you can finitely approximate the feature map by Taylor expansion.


In [72]:
import torch
import math

In [64]:
def nu_quadratic_kernel(x, x_prime):
    """Compute the quadratic kernel between two arrays of coordinates.

    Parameters
    ----------
    x : array-like, shape (n_features)
        First array of coordinates.
    x_prime : array-like, shape (n_features)
        Second array of coordinates.

    Returns
    -------
    k : array-like, shape (1)
        Kernel value.
    """
    ret = torch.dot(x, x_prime)+torch.tensor(1.0)
    return ret ** 2
    

In [65]:
nu_quadratic_kernel(torch.tensor([1, 2, 3]), torch.tensor([4, 5, 6]))

tensor(1089.)

In [66]:
def nu_quadratic_feature_map(x):
    """Compute the feature map corresponding to the quadratic kernel.

    Parameters
    ----------
    x : array-like, shape (n_features)
        Array of coordinates.

    Returns
    -------
    phi_x : array-like, shape (n_features)
        Feature map.
    """
    d = len(x)
    x = x.flip(0)
    list_map = [e**2 for e in x]
    c = torch.sqrt(torch.tensor(2))
    for i in range(d):
        for j in range(i+1,d):
            list_map.append(c*x[i]*x[j])
    list_map.extend([c*e for e in x])

    
    return torch.tensor(list_map)

In [67]:
# Check that the two functions are equivalent on a randomly-initialized array
x = torch.tensor([1, 2, 3])
y = torch.tensor([4, 5, 6])
print(nu_quadratic_kernel(x, y))
print(torch.dot(nu_quadratic_feature_map(x),nu_quadratic_feature_map(y)))

tensor(1089.)
tensor(1088.)


In [246]:
def gaussian_kernel(x, x_prime, sigma):
    """Compute the Gaussian kernel between two arrays of coordinates.

    Parameters
    ----------
    x : array-like, shape (n_features)
        First array of coordinates.
    x_prime : array-like, shape (n_features)
        Second array of coordinates.
    sigma : float
        Kernel standard deviation.

    Returns
    -------
    k : array-like, shape (1)
        Kernel value.
    """
    v =  x-x_prime
    v = torch.sum(v**2)
    
    return torch.exp(-v/(2*sigma**2))


In [262]:
def gaussian_feature_map(x, sigma, order):
    """Compute the feature map corresponding to the Gaussian kernel.

    Parameters
    ----------
    x : array-like, shape (n_features)
        Array of coordinates.
    sigma : float
        Kernel standard deviation.

    Returns
    -------
    phi_x : array-like, shape (n_features)
        Feature map.
    """
    first_factor = torch.exp(-torch.dot(x.transpose(-1,0),x)/(2*sigma**2))
    ret = torch.stack([((x/torch.sqrt(torch.tensor(sigma)))**i/torch.sqrt(torch.tensor(math.factorial(i)))) for i in range(0,order)])
    ret *= first_factor
    return ret.flatten()
    
    


In [263]:
x = torch.tensor([1, 2, 3])
y = torch.tensor([4, 5, 6])
c = torch.sqrt(torch.tensor(2))
print(gaussian_kernel(x, y, c))
print(torch.dot(gaussian_feature_map(x, c, 20), gaussian_feature_map(y, 1, 20)))


tensor(0.0012)
tensor(1.8709e-12)


/tmp/ipykernel_1044/2888552204.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ret = torch.stack([((x/torch.sqrt(torch.tensor(sigma)))**i/torch.sqrt(torch.tensor(math.factorial(i)))) for i in range(0,order)])
